In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [89]:
data=pd.read_csv(r"C:\Users\pc\Downloads\fake_news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [91]:
data.shape

(20800, 5)

In [93]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [95]:
data.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [97]:
data=data.drop(['id'],axis=1)

In [99]:
data=data.fillna('')

In [101]:
data['content']=data['author']+''+data['title']+''+data['text']

In [103]:
data=data.drop(['title','author','text'],axis=1)

In [105]:
data.head()

,label,content
0,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,1,Consortiumnews.comWhy the Truth Might Get You ...
3,1,Jessica Purkiss15 Civilians Killed In Single U...
4,1,Howard PortnoyIranian woman jailed for fiction...


In [107]:
data['content']=data['content'].apply(lambda x:"".join(x.lower()for x in x.split()))

In [109]:
data['content']=data['content'].str.replace('[^\w\s]','')

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\pc\AppData\Local\Temp\ipykernel_14836\2466547887.py:1: SyntaxWarning: invalid escape sequence '\w'
  data['content']=data['content'].str.replace('[^\w\s]','')


In [111]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [113]:
from nltk.corpus import stopwords
stop=stopwords.words('english')
data['content']=data['content'].apply(lambda x:"".join(x for x in x.split() if x not in stop))

In [115]:
import pandas as pd
from textblob import Word

data['content'] = data['content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['content'].head()


0    darrelllucushousedemaide:wedidn’tevenseecomey’...
1    danielj.flynnflynn:hillaryclinton,bigwomanonca...
2    consortiumnews.comwhythetruthmightgetyoufiredw...
3    jessicapurkiss15civilianskilledinsingleusairst...
4    howardportnoyiranianwomanjailedforfictionalunp...
Name: content, dtype: object

In [117]:
x=data[['content']]
y=data['label']

In [119]:
from sklearn.model_selection import train_test_split

In [121]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=45,stratify=y)

In [123]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(14560, 1)
(14560,)
(6240, 1)
(6240,)


In [125]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=500)

tfidf_vect.fit(data['content'])

xtrain_tfidf = tfidf_vect.transform(x_train['content'])
xtest_tfidf = tfidf_vect.transform(x_test['content'])

In [135]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
pclf=PassiveAggressiveClassifier()
pclf.fit(xtrain_tfidf,y_train)
predictions=pclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87      3116
           1       0.87      0.86      0.87      3124

    accuracy                           0.87      6240
   macro avg       0.87      0.87      0.87      6240
weighted avg       0.87      0.87      0.87      6240



In [137]:
print(metrics.confusion_matrix(y_test,predictions))

[[2729  387]
 [ 443 2681]]


In [139]:
from sklearn.neural_network import MLPClassifier
mlpclf=MLPClassifier(hidden_layer_sizes=(256,64,16),
                     activation='relu',
                     solver='adam')
mlpclf.fit(xtrain_tfidf,y_train)
predictions=mlpclf.predict(xtest_tfidf)
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      3116
           1       0.87      0.90      0.89      3124

    accuracy                           0.89      6240
   macro avg       0.89      0.89      0.89      6240
weighted avg       0.89      0.89      0.89      6240



In [143]:
print(metrics.confusion_matrix(y_test,predictions))

[[2706  410]
 [ 301 2823]]


In [147]:
import pickle
pickle.dump(mlpclf,open("fakenews1.pkl","wb"))